# Initial Analysis

#### Imports

In [1]:
import sys
import math
import warnings

import psycopg2
import wrds
import gzip

import seaborn as sns
import os
import quandl
import json
import zipfile
import matplotlib.pyplot as plt
import matplotlib.dates as mdates

import functools
import requests
import io

import urllib.request
from urllib.error import HTTPError
from html_table_parser.parser import HTMLTableParser
from bs4 import BeautifulSoup
import re

import plotnine as p9
from plotnine import ggplot, scale_x_date, guides, guide_legend, geom_bar, scale_y_continuous, \
    scale_color_identity, geom_line, geom_point, labs, theme_minimal, theme, element_blank, element_text, \
        geom_ribbon, geom_hline, aes, scale_size_manual, scale_color_manual, ggtitle

from datetime import datetime
import datetime

import pandas as pd
import pandas_market_calendars as mcal
from pandas.plotting import autocorrelation_plot
import numpy as np
from numpy import cumsum, log, polyfit, sqrt, std, subtract
import scipy as sp
from scipy.stats import norm
import scipy.stats as stats

from statsmodels.tsa.stattools import coint
from statsmodels.graphics.tsaplots import plot_acf
import statsmodels.api as sm
from statsmodels.tsa.stattools import acf
from statsmodels.regression.linear_model import OLS
from statsmodels.tools.tools import add_constant

from collections import deque
from bisect import insort, bisect_left
from itertools import islice


### Retrieve as Variable

In [2]:
csv_file_path = 'option_data.csv'
spydata = pd.read_csv(csv_file_path)

In [3]:
display(spydata)

,date,exdate,last_date,cp_flag,strike_price,best_bid,best_offer,volume,open_interest,impl_volatility,delta,gamma,vega,theta,expiry_indicator
0,2018-01-02,2018-01-03,2017-12-28,C,235000.0,33.59,33.81,0.0,187.0,NaN,NaN,NaN,NaN,NaN,w
1,2018-01-02,2018-01-03,2018-01-02,C,240000.0,28.59,28.76,1.0,88.0,NaN,NaN,NaN,NaN,NaN,w
2,2018-01-02,2018-01-03,2017-12-27,C,242500.0,26.09,26.32,0.0,2.0,NaN,NaN,NaN,NaN,NaN,w
3,2018-01-02,2018-01-03,2018-01-02,C,245000.0,23.59,23.81,12.0,58.0,NaN,NaN,NaN,NaN,NaN,w
4,2018-01-02,2018-01-03,NaN,C,247500.0,21.08,21.32,0.0,0.0,NaN,NaN,NaN,NaN,NaN,w
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10448189,2023-02-28,2025-03-21,2022-12-28,P,600000.0,200.50,205.50,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN
10448190,2023-02-28,2025-03-21,NaN,P,605000.0,205.50,210.50,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN
10448191,2023-02-28,2025-03-21,NaN,P,610000.0,210.00,215.00,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN
10448192,2023-02-28,2025-03-21,NaN,P,615000.0,215.00,220.00,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN


Order by the appropriate date, then expiry date, then strike price within the date spread.

In [4]:
spydata_sorted = spydata.sort_values(by=['date', 'exdate', 'cp_flag', 'strike_price'])
#spydata_cleaned = spydata_sorted.dropna(subset=['impl_volatility', 'delta', 'gamma', 'vega', 'theta'])
#spydata_final = spydata_cleaned.reset_index(drop=True)
spydata_final = spydata_sorted.reset_index(drop=True)

In [5]:
display(spydata_final)

,date,exdate,last_date,cp_flag,strike_price,best_bid,best_offer,volume,open_interest,impl_volatility,delta,gamma,vega,theta,expiry_indicator
0,2018-01-02,2018-01-03,2017-12-28,C,235000.0,33.59,33.81,0.0,187.0,NaN,NaN,NaN,NaN,NaN,w
1,2018-01-02,2018-01-03,2018-01-02,C,240000.0,28.59,28.76,1.0,88.0,NaN,NaN,NaN,NaN,NaN,w
2,2018-01-02,2018-01-03,2017-12-27,C,242500.0,26.09,26.32,0.0,2.0,NaN,NaN,NaN,NaN,NaN,w
3,2018-01-02,2018-01-03,2018-01-02,C,245000.0,23.59,23.81,12.0,58.0,NaN,NaN,NaN,NaN,NaN,w
4,2018-01-02,2018-01-03,NaN,C,247500.0,21.08,21.32,0.0,0.0,NaN,NaN,NaN,NaN,NaN,w
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10448189,2023-02-28,2025-03-21,2022-12-28,P,600000.0,200.50,205.50,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN
10448190,2023-02-28,2025-03-21,NaN,P,605000.0,205.50,210.50,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN
10448191,2023-02-28,2025-03-21,NaN,P,610000.0,210.00,215.00,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN
10448192,2023-02-28,2025-03-21,NaN,P,615000.0,215.00,220.00,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN


In [21]:
spydata_final.to_csv('sorteddata.csv', index=False)

Rather than download again, simply reload.

In [2]:
spy_optiondata = pd.read_csv('sorteddata.csv')

In [3]:
display(spy_optiondata)

,date,exdate,last_date,cp_flag,strike_price,best_bid,best_offer,volume,open_interest,impl_volatility,delta,gamma,vega,theta,expiry_indicator
0,2018-01-02,2018-01-03,2017-12-28,C,235000.0,33.59,33.81,0.0,187.0,NaN,NaN,NaN,NaN,NaN,w
1,2018-01-02,2018-01-03,2018-01-02,C,240000.0,28.59,28.76,1.0,88.0,NaN,NaN,NaN,NaN,NaN,w
2,2018-01-02,2018-01-03,2017-12-27,C,242500.0,26.09,26.32,0.0,2.0,NaN,NaN,NaN,NaN,NaN,w
3,2018-01-02,2018-01-03,2018-01-02,C,245000.0,23.59,23.81,12.0,58.0,NaN,NaN,NaN,NaN,NaN,w
4,2018-01-02,2018-01-03,NaN,C,247500.0,21.08,21.32,0.0,0.0,NaN,NaN,NaN,NaN,NaN,w
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10448189,2023-02-28,2025-03-21,2022-12-28,P,600000.0,200.50,205.50,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN
10448190,2023-02-28,2025-03-21,NaN,P,605000.0,205.50,210.50,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN
10448191,2023-02-28,2025-03-21,NaN,P,610000.0,210.00,215.00,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN
10448192,2023-02-28,2025-03-21,NaN,P,615000.0,215.00,220.00,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN


Now we'll load the ticker price data, directly from local.

In [4]:
spy_tickerdata = pd.read_csv('spy_tickerdata.csv')

In [5]:
display(spy_tickerdata)

,ticker,date,open,high,low,close,volume,dividend,split,adj_open,adj_high,adj_low,adj_close,adj_volume
0,SPY,1993-08-13,45.093700,45.156200,45.093700,45.125000,103500.0,0.0,1.0,25.778428,25.814157,25.778428,25.796321,103500.0
1,SPY,1993-08-16,45.156200,45.500000,45.156200,45.375000,241800.0,0.0,1.0,25.814157,26.010695,25.814157,25.939237,241800.0
2,SPY,1993-08-17,45.343700,45.531200,45.343700,45.531200,369300.0,0.0,1.0,25.921344,26.028531,25.921344,26.028531,369300.0
3,SPY,1993-08-18,45.687500,45.875000,45.656200,45.781200,414300.0,0.0,1.0,26.117882,26.225069,26.099989,26.171447,414300.0
4,SPY,1993-08-19,45.812500,45.812500,45.718700,45.781200,28500.0,0.0,1.0,26.189340,26.189340,26.135718,26.171447,28500.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7814,SPY,1993-06-22,44.656200,44.656200,44.562500,44.625000,137500.0,0.0,1.0,25.528325,25.528325,25.474761,25.510490,137500.0
7815,SPY,1994-11-16,46.765598,46.843700,46.609299,46.843700,106900.0,0.0,1.0,27.589587,27.635663,27.497377,27.635663,106900.0
7816,SPY,2010-03-10,114.510002,115.279999,114.410004,114.970001,186088800.0,0.0,1.0,87.964535,88.556033,87.887719,88.317898,186088800.0
7817,SPY,2022-02-15,443.730000,446.280000,443.180000,446.100000,88435136.0,0.0,1.0,430.043935,432.515285,429.510899,432.340837,88435136.0


## Adjustments

Make some adjustments and attach to our data table for options.

In [6]:
spy_tickerdata['date'] = pd.to_datetime(spy_tickerdata['date'])
start_date = '2018-01-01'
end_date = '2023-02-28'
spy_tickerdata_filtered = spy_tickerdata[(spy_tickerdata['date'] >= start_date) & (spy_tickerdata['date'] <= end_date)]

spy_tickerdata_sorted = spy_tickerdata_filtered.sort_values(by='date')
spy_tickerdata_reduced = spy_tickerdata_sorted[['date', 'adj_open', 'adj_high', 'adj_low', 'adj_close', 'adj_volume']]

In [7]:
spy_optiondata['date'] = pd.to_datetime(spy_optiondata['date'])
spy_optiondata['strike_price'] = spy_optiondata['strike_price'] / 1000.0

In [8]:
spy_optiondata_enriched = pd.merge(spy_optiondata, spy_tickerdata_reduced, on='date', how='left')

In [9]:
display(spy_optiondata_enriched)

,date,exdate,last_date,cp_flag,strike_price,best_bid,best_offer,volume,open_interest,impl_volatility,delta,gamma,vega,theta,expiry_indicator,adj_open,adj_high,adj_low,adj_close,adj_volume
0,2018-01-02,2018-01-03,2017-12-28,C,235.0,33.59,33.81,0.0,187.0,NaN,NaN,NaN,NaN,NaN,w,242.053393,242.930005,241.655754,242.893856,86655749.0
1,2018-01-02,2018-01-03,2018-01-02,C,240.0,28.59,28.76,1.0,88.0,NaN,NaN,NaN,NaN,NaN,w,242.053393,242.930005,241.655754,242.893856,86655749.0
2,2018-01-02,2018-01-03,2017-12-27,C,242.5,26.09,26.32,0.0,2.0,NaN,NaN,NaN,NaN,NaN,w,242.053393,242.930005,241.655754,242.893856,86655749.0
3,2018-01-02,2018-01-03,2018-01-02,C,245.0,23.59,23.81,12.0,58.0,NaN,NaN,NaN,NaN,NaN,w,242.053393,242.930005,241.655754,242.893856,86655749.0
4,2018-01-02,2018-01-03,NaN,C,247.5,21.08,21.32,0.0,0.0,NaN,NaN,NaN,NaN,NaN,w,242.053393,242.930005,241.655754,242.893856,86655749.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10448189,2023-02-28,2025-03-21,2022-12-28,P,600.0,200.50,205.50,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,391.240559,393.259649,390.176843,390.285185,96141367.0
10448190,2023-02-28,2025-03-21,NaN,P,605.0,205.50,210.50,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,391.240559,393.259649,390.176843,390.285185,96141367.0
10448191,2023-02-28,2025-03-21,NaN,P,610.0,210.00,215.00,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,391.240559,393.259649,390.176843,390.285185,96141367.0
10448192,2023-02-28,2025-03-21,NaN,P,615.0,215.00,220.00,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,391.240559,393.259649,390.176843,390.285185,96141367.0


In [10]:
spy_optiondata_enriched.to_csv('combinedata.csv', index=False)

In [11]:
spydata_cleaned = spy_optiondata_enriched.dropna(subset=['impl_volatility', 'delta', 'gamma', 'vega', 'theta'])
spydata_dropna = spydata_cleaned.reset_index(drop=True)
spydata_dropna.to_csv('combinedata_dropna.csv', index=False)

## Load useful data

In [ ]:
filepath = 'combinedata.csv'
filepath1 = 'combinedata_dropna.csv'
options_df = pd.read_csv(filepath)

In [ ]:
display(options_df)